In [15]:
# Import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_extract


In [2]:
# Create a Spark Session
spark = SparkSession.builder\
    .appName("Automotive Chip Reliability ETL")\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/02 16:11:51 WARN Utils: Your hostname, Evans-MacBook-Pro-2.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.149 instead (on interface en0)
25/06/02 16:11:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/02 16:11:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load the CSV file into a Spark DataFrame
raw_chip_df = spark.read.csv('data/automotive_chips_reliability_dataset.csv', header=True, inferSchema=True)

Row(Chip ID='Chip001', Technology Node=5, FSDOI Type='Thick Layer', Gate Density=989, Substrate Type='Silicon', Power Consumption=3.12, Thermal Dispersion Efficiency=70.364436513031, Mechanical Stress Resistance=195.48182400614584, Temperature Range='-20°C to 85°C', Humidity Levels=80, Vibration Levels='Medium', External Load Stress=48, Thermal Stress=124.04184890445367, Mechanical Stress=219.88458531609044, Electrical Density=49.10287370072312, Electromigration Risk=0.03273841845647966, Deformation=0.4980688861428193, Thermal Test Result=33.00966001869167, Mechanical Stress Test Result=145.85045256562228, Failure Mode='Thermal Failure', Failure Rate (%)=4.069810220267985, MTTF (hours)=2482, Reliability Score=7.033093793285942, Failure Rate Comparison='2x Reduced', Statistical Significance='p-value < 0.05')

In [5]:
raw_chip_df.printSchema()

root
 |-- Chip ID: string (nullable = true)
 |-- Technology Node: integer (nullable = true)
 |-- FSDOI Type: string (nullable = true)
 |-- Gate Density: integer (nullable = true)
 |-- Substrate Type: string (nullable = true)
 |-- Power Consumption: double (nullable = true)
 |-- Thermal Dispersion Efficiency: double (nullable = true)
 |-- Mechanical Stress Resistance: double (nullable = true)
 |-- Temperature Range: string (nullable = true)
 |-- Humidity Levels: integer (nullable = true)
 |-- Vibration Levels: string (nullable = true)
 |-- External Load Stress: integer (nullable = true)
 |-- Thermal Stress: double (nullable = true)
 |-- Mechanical Stress: double (nullable = true)
 |-- Electrical Density: double (nullable = true)
 |-- Electromigration Risk: double (nullable = true)
 |-- Deformation: double (nullable = true)
 |-- Thermal Test Result: double (nullable = true)
 |-- Mechanical Stress Test Result: double (nullable = true)
 |-- Failure Mode: string (nullable = true)
 |-- Failu

In [6]:
raw_chip_df.show(5)

+-------+---------------+-----------+------------+-----------------+-----------------+-----------------------------+----------------------------+-----------------+---------------+----------------+--------------------+------------------+------------------+------------------+---------------------+-------------------+-------------------+-----------------------------+------------------+-----------------+------------+-----------------+-----------------------+------------------------+
|Chip ID|Technology Node| FSDOI Type|Gate Density|   Substrate Type|Power Consumption|Thermal Dispersion Efficiency|Mechanical Stress Resistance|Temperature Range|Humidity Levels|Vibration Levels|External Load Stress|    Thermal Stress| Mechanical Stress|Electrical Density|Electromigration Risk|        Deformation|Thermal Test Result|Mechanical Stress Test Result|      Failure Mode| Failure Rate (%)|MTTF (hours)|Reliability Score|Failure Rate Comparison|Statistical Significance|
+-------+---------------+-------

In [7]:
raw_chip_df.columns

['Chip ID',
 'Technology Node',
 'FSDOI Type',
 'Gate Density',
 'Substrate Type',
 'Power Consumption',
 'Thermal Dispersion Efficiency',
 'Mechanical Stress Resistance',
 'Temperature Range',
 'Humidity Levels',
 'Vibration Levels',
 'External Load Stress',
 'Thermal Stress',
 'Mechanical Stress',
 'Electrical Density',
 'Electromigration Risk',
 'Deformation',
 'Thermal Test Result',
 'Mechanical Stress Test Result',
 'Failure Mode',
 'Failure Rate (%)',
 'MTTF (hours)',
 'Reliability Score',
 'Failure Rate Comparison',
 'Statistical Significance']

In [10]:
# Before we verify the exact meaning of these columns and what they 
# represent, we run some typical/standard transformations 

# Standardize column names
raw_chip_df = raw_chip_df.select([col(c).alias(c.lower()\
    .replace(" ", "_").replace("(", "").replace(")", "")) \
        for c in raw_chip_df.columns])

In [13]:
# Handle missing values
raw_chip_df.select([F.count(F.when(F.col(c).isNull(), \
    c)).alias(c) for c in raw_chip_df.columns]).show()

+-------+---------------+----------+------------+--------------+-----------------+-----------------------------+----------------------------+-----------------+---------------+----------------+--------------------+--------------+-----------------+------------------+---------------------+-----------+-------------------+-----------------------------+------------+--------------+----------+-----------------+-----------------------+------------------------+
|chip_id|technology_node|fsdoi_type|gate_density|substrate_type|power_consumption|thermal_dispersion_efficiency|mechanical_stress_resistance|temperature_range|humidity_levels|vibration_levels|external_load_stress|thermal_stress|mechanical_stress|electrical_density|electromigration_risk|deformation|thermal_test_result|mechanical_stress_test_result|failure_mode|failure_rate_%|mttf_hours|reliability_score|failure_rate_comparison|statistical_significance|
+-------+---------------+----------+------------+--------------+-----------------+------

In [16]:
# Split or Parse Composite Columns
raw_chip_df = raw_chip_df.withColumn("temp_min", regexp_extract\
    ("temperature_range", r"(-?\d+)", 1).cast("int"))
raw_chip_df = raw_chip_df.withColumn("temp_max", regexp_extract\
    ("temperature_range", r"to\s*(-?\d+)", 1).cast("int"))

In [18]:
raw_chip_df.printSchema()

root
 |-- chip_id: string (nullable = true)
 |-- technology_node: integer (nullable = true)
 |-- fsdoi_type: string (nullable = true)
 |-- gate_density: integer (nullable = true)
 |-- substrate_type: string (nullable = true)
 |-- power_consumption: double (nullable = true)
 |-- thermal_dispersion_efficiency: double (nullable = true)
 |-- mechanical_stress_resistance: double (nullable = true)
 |-- temperature_range: string (nullable = true)
 |-- humidity_levels: integer (nullable = true)
 |-- vibration_levels: string (nullable = true)
 |-- external_load_stress: integer (nullable = true)
 |-- thermal_stress: double (nullable = true)
 |-- mechanical_stress: double (nullable = true)
 |-- electrical_density: double (nullable = true)
 |-- electromigration_risk: double (nullable = true)
 |-- deformation: double (nullable = true)
 |-- thermal_test_result: double (nullable = true)
 |-- mechanical_stress_test_result: double (nullable = true)
 |-- failure_mode: string (nullable = true)
 |-- failu